<a href="https://colab.research.google.com/github/Weff24/PrivacyPlate/blob/main/pp_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Notebook Setups

## 1.1 Install Environment Requirements

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -U yolov5

In [4]:
import cv2
import pytesseract
import yolov5

pytesseract.pytesseract.tesseract_cmd = ( r'/usr/bin/tesseract' )

## 1.2 Setup Methods

In [5]:
### Extraction Method ###
def extract_license_plate_text(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding or other preprocessing techniques if needed
    # Example: gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # Use Tesseract OCR to extract text
    text = pytesseract.image_to_string(gray, config='--psm 7 --oem 1')

    return text

### Loading Model For Yolo ###
def load_model_bounding():
  model = yolov5.load('keremberke/yolov5m-license-plate')

  # set model parameters
  model.conf = 0.5  # NMS confidence threshold
  model.iou = 0.45  # NMS IoU threshold
  model.agnostic = False  # NMS class-agnostic
  model.multi_label = False  # NMS multiple labels per box
  model.max_det = 1000  # maximum number of detections per image

  return model

### Show Bounding Boxes Using Yolo ###
def bounding_box_show(image_path, model):
  # perform inference
  results = model(image_path, size=640)

  # inference with test time augmentation
  results = model(image_path, augment=True)

  # parse results
  predictions = results.pred[0]
  boxes = predictions[:, :4] # x1, y1, x2, y2
  scores = predictions[:, 4]
  categories = predictions[:, 5]

  # show detection bounding boxes on image
  results.show()

# 2. Image Blurring

# 3. Pipeline Evaluation

In [8]:
bounding_box_model = load_model_bounding()

while True:
  # 1. Ask for user input of the image path
  img_path = input("Please enter your desired image (q to quit)")
  if img_path == "q":
    print("Exiting...")
    break

  extracted_text = extract_license_plate_text(img_path)

  # 1. Extracted text first
  if len(extracted_text) != 0:
    print(f"Extracted text from - {img_path}: {extracted_text}")
    continue

  # 2. Run YOLO model to show bounding box if the extracted text is empty
  bounding_box_show(img_path, bounding_box_model)



Please enter your desired image (q to quit)test.png
Extracted text from - test.png: EPGeMN112]

Please enter your desired image (q to quit)q
Exiting...
